## Step 1: Import some packages 
As in often the case, we start our code by importing some Python modules. 

Remember: **Your code will not work** unless you run the cell in which the modules are imported.

In [ ]:
# import python modules to use for analysis
import numpy as np
import pandas as pd
import os
from IPython.utils import io
import glob
import matplotlib.pyplot as plt
import ast
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# install package for fitting
install_psignifit = True

if install_psignifit:
    # stuff for fitting pphys
    # insert at 1, 0 is the script path (or '' in REPL)
    with io.capture_output() as temp:
        !pip install https://github.com/wichmann-lab/python-psignifit/zipball/master

import psignifit as ps

## Step 2: Define some functions
In the cell below we define two functions
    
    fit_ps
    
    plot_ps

that will fit fit the psychometric function and plot the data, respectively. You do not need to worry about the internal workings of those functions, although you are welcome to explore them if you are interested. 

Remember: **Your code will not work** unless you run the cell in which the functions are defined first. 

In [ ]:
def fit_ps(stim_params, fit_data, n_trials, options=dict()):
    fit_data = np.vstack([np.array(stim_params), np.array(fit_data)*np.array(n_trials), np.array(n_trials)])
    if not options:
        options = dict();   # initialize as an empty dictionary
        options['sigmoidName'] = 'norm';   # choose a cumulative Gauss as the sigmoid  
        options['expType']     = 'equalAsymptote';   # choose 2-AFC as the experiment type  
                                   # this sets the guessing rate to .5 (fixed) and  
                                   # fits the rest of the parameters
    with io.capture_output() as captured:
        temp_params = ps.psignifit(fit_data.transpose(),options)
    threshold = ps.getThreshold(temp_params,.5)[0]
    slope = ps.getSlopePC(temp_params,.5)
    return temp_params, threshold, slope

def plot_ps(info_df, fit_params, participant, annotate=False, plot_function=False):
    # plot data from two experiments
    condition_list = ["same", "small"]
    
    if not participant in ["average", "averages", "mean", "means"]:
        if participant in info_df['ID'].to_list():
            cur_df = info_df.loc[info_df['ID'] == participant]
            idx = info_df[info_df['ID'] == participant].index[0]
            cur_params = [ x[idx-1] for x in fit_params ] 
            # use pse and slope from individual subject
            cur_pse = [cur_df["pse-{0}".format(c)].to_list()[0] for c in condition_list ]
            cur_slope = [cur_df["slope-{0}".format(c)].to_list()[0] for c in condition_list ]
        else:
            print("Unknown participant! Try again!")
            return
    else:
        cur_df = info_df
        cur_params = [ x[-1] for x in fit_params ]
        # use pse and slope from average fit
        cur_pse = [cur_df["pse-{0}-avefit".format(c)].to_list()[0] for c in condition_list ]
        cur_slope = [cur_df["slope-{0}-avefit".format(c)].to_list()[0] for c in condition_list ]

    ps_kwargs = dict(plotData=False, showImediate=False, plotAsymptote=False)
    eb_kwargs = dict(linestyle='none', mfc='none', ms=8, clip_on=False)

    fig, ax = plt.subplots(1,2, figsize=[15,5])
    for t, data_type in enumerate(["rt", "testbigger"]):
        for c, cond in enumerate(condition_list):
            means = [cur_df["{0}-{1}-{2}".format(data_type, cond, str(x))].mean() for x in test_inner_sizes]
            stddev = [cur_df["{0}-{1}-{2}".format(data_type, cond, str(x))].std() for x in test_inner_sizes]
            n = [cur_df["{0}-{1}-{2}".format(data_type, cond, str(x))].count() for x in test_inner_sizes]
            stderr = [i / np.sqrt(j) for i, j in zip(stddev, n)]
            ci = [x * 1.96 for x in stderr]
            if cond is 'same':
                eb_kwargs["marker"]='o'
            else:
                eb_kwargs["marker"]='s'
            e_h = ax[t].errorbar(test_inner_sizes, means, yerr=ci, label=condition_list[c]+ " inducers", **eb_kwargs)
            ax[t].set_xticks(test_inner_sizes)

            if data_type == 'rt':
                ax[t].set_ylim([0,5000])
            else:
                ax[t].set_ylim([0,1])
            if "testbigger" in data_type:
                title_str = 'Responses'
                ylabel_str = "test > ref"
                many_xx = np.linspace(20, 30)
                if plot_function:
                    ps.psigniplot.plotPsych(cur_params[c], axisHandle=ax[t],lineColor=e_h[0].get_color(), **ps_kwargs)
                ax[t].plot((10, 40), (0.5, 0.5), 'k:')
                
                if annotate:
                    if cond is "same":
                        ax[t].text(26, 0.40, "PSE = {:.2f}, slope = {:.2f}".format(cur_pse[c], cur_slope[c]), color=e_h[0].get_color())
                    else:
                        ax[t].text(26, 0.45, "PSE = {:.2f}, slope = {:.2f}".format(cur_pse[c], cur_slope[c]), color=e_h[0].get_color())
            else:
                title_str = 'Reaction Time'
                ylabel_str = "RT (ms)"
            ax[t].set_xlim([19,31])  

            ax[t].set_xlabel("inner test size (ref: 25)")
            ax[t].set_ylabel(ylabel_str)
            ax[t].title.set_text(title_str)
            ax[t].legend()

## Step 3: Get the data from Open Science Foundation
The data files are shared in an public repository at the [Open Science Foundation](https://osf.io/s6mxd/]). The code below simply grabs the data and makes it accessible to the current workbook.

In [ ]:
!osf --project s6mxd clone

## Step 4: Load and organize the data from each participant

The code below loads in the individual data files in csv format that each participant, including you, created when they did the experiment. When running an experiment online, you would normally have some sort of back-end that saves the files automatically somewhere, so you do not have to trust your participant to send you the raw data. 

In [ ]:
data_dir = ["s6mxd/osfstorage/data"]

# define empty lists that will hold the number of subjects, rejected subjects and test subjects
sub_count = [0]*len(data_dir)    # included datasets
reject_count = [0]*len(data_dir) # complete datasets, but rejected due to performance
test_count = [0]*len(data_dir)   # incomplete test datasets

complete_subs = [] # subjects that produced complete datasets

prac_n = 16 # number of practice trials
exp_n = 160 # number of experiment trials

info_df = pd.DataFrame()

for e, cur_dir in enumerate(data_dir):
    file_list = glob.glob(cur_dir + "/**/psyc4260*.csv") + glob.glob(cur_dir + "/**/nrsc2200*.csv")
    file_list.sort()
    exp_subs = [] # list to hold the subjects in this experiment
    for file in file_list:
        # load the data
        try:
            sub_data = pd.read_csv(file)
            if "trial_type" not in sub_data:
                sub_data = pd.read_csv(file, skiprows=1)
            # in the past, some trials were duplicated in the data file, the code below takes care of that
            sub_data = sub_data[sub_data['trial_index'].apply(lambda x: str(x).isdigit())]
            sub_data = sub_data.drop_duplicates()
        except:
            print("Failed to load file {0}".format(file.split(cur_dir)[1]))
            
        # get id
        try:
            survey_resp = sub_data[sub_data["trial_type"]=="survey-html-form"].loc[0]["response"]
            survey_resp = survey_resp.replace(':"}',':""}')
            sub_info = ast.literal_eval(survey_resp)
        except:
            sub_info = {}
        # see if id was stored
        if 'p_id' in sub_info.keys():
            # if participant ID was recorded, use it to get rid of data with test subject ID 214984
            # this is the ID associated with URPP's sample link
            try:    
                sub_id = sub_info["p_id"]
            except:
                sub_id = "nan"
            del sub_info['p_id']
            if sub_id in ["214984", "603921b616b3893be91674d1"]:
                test_count[e] = test_count[e] + 1
                continue
        else:
            sub_id = "nan"
        # do quality control on the data
        if sub_data.shape[1] < 10:
            print(e, sub_id, "incomplete file, shape:{0}x{1}".format(sub_data.shape[0],sub_data.shape[1]))
            test_count[e] = test_count[e] + 1 # record test subject to the test_count, by adding 1 at the relevant position
            continue
        
        # now skip if this subjects data is already in the set
        if sub_id in exp_subs:
            continue
        
        # add participant to list of subjects for this experiment
        exp_subs.append(sub_id)
        
        # start populating sub_info dict: 
        sub_info["experiment"] = e
        sub_info["ID"] = sub_id
        
        # get course and timing for this subject
        course = file.split('-')[0].split('/')[-1].upper()
        
        try:
            dt = datetime.strptime(sub_data["recorded_at"][0], '%Y-%m-%d %H:%M:%S')
        except:
            dt = datetime.strptime(time.ctime(os.path.getmtime(file)), '%a %b %d %H:%M:%S %Y')
            
        if dt.month < 5:
            sub_info["course"] = "{}_W{}".format(course, dt.year)
        elif dt.month > 8:
            sub_info["course"] = "{}_F{}".format(course, dt.year)
        else:
            sub_info["course"] = "{}_S{}".format(course, dt.year)

        # Cognition.run specific error: replace strings with logicals
        sub_data = sub_data.replace("true", True)
        sub_data = sub_data.replace("false", False)
        
        pphys_data = sub_data[sub_data["trial_type"] == "psychophysics"]
        pphys_data = pphys_data.replace(np.nan, "")
        pphys_data = pphys_data.replace('"', "")
        pphys_data = pphys_data[pphys_data.test_inner_radius != ""]
        pphys_data = pphys_data.astype(
            {"test_inner_radius": 'int', "test_outer_radius": 'int', "ref_inner_radius": 'int', "ref_outer_radius": 'int' })
        
        test_inner_sizes = [x for x in pphys_data["test_inner_radius"].unique()]
        test_inner_sizes.sort()
        
        test_outer_sizes = [x for x in pphys_data["test_outer_radius"].unique()]
        test_outer_sizes.sort()
        
        ref_outer_size = [x for x in pphys_data["ref_outer_radius"].unique()][0]
        ref_inner_size = [x for x in pphys_data["ref_inner_radius"].unique()][0]
        
        rt_reject = False
        acc_reject = False
        for size in test_inner_sizes:
            for o in test_outer_sizes:
                cur_data = pphys_data[(pphys_data["test_inner_radius"]==size) & (pphys_data["test_outer_radius"]==o)]
                # only include trials with RTs less than 5 secs
                cur_data = cur_data[cur_data["rt"] < 3000]
                if len(cur_data) < 10:
                    rt_reject = True
                if o == ref_outer_size:
                    sub_info["rt-same-"+str(size)] = cur_data["rt"].mean()
                    temp_bigger = cur_data["test_bigger"]
                    sub_info["testbigger-same-" + str(size) ] = np.nansum(temp_bigger)/len(temp_bigger)
                    sub_info["ntrials-same-" + str(size) ] = len(temp_bigger)
                    if size == test_inner_sizes[0]:
                        if np.nansum(temp_bigger)/len(temp_bigger) > 0.2:
                            acc_reject = True
                    if size == test_inner_sizes[-1]:
                        if np.nansum(temp_bigger)/len(temp_bigger) < 0.8:
                            acc_reject = True
                else:
                    sub_info["rt-small-"+str(size)] = cur_data["rt"].mean()
                    temp_bigger = cur_data["test_bigger"]
                    sub_info["testbigger-small-" + str(size) ] = np.nansum(temp_bigger)/len(temp_bigger)
                    sub_info["ntrials-small-" + str(size) ] = len(temp_bigger)
        if acc_reject:
            print("Excluding {} : {}, chance performance".format(sub_info["course"], sub_id))
            continue 
        elif rt_reject:
            print("Excluding {} : {}, less than 10 trials with RT < 3 secs".format(sub_info["course"], sub_id))
            continue
        sub_count[e] = sub_count[e]+1
        sub_info["count"] = sub_count[e]
        info_df = pd.concat([info_df, pd.json_normalize(sub_info)], sort=False)

        # add to dataframe of complete subjects 
        complete_subs.append(exp_subs)
info_df = info_df.set_index("count")

cols = info_df.columns.tolist()
cols.sort()
cols.insert(0, cols.pop(cols.index("experiment")))
if "age" in cols:
    cols.insert(2, cols.pop(cols.index("age")))
    cols.insert(3, cols.pop(cols.index("handedness")))
    cols.insert(4, cols.pop(cols.index("sex")))
    cols.insert(5, cols.pop(cols.index("other_sex")))
info_df = info_df[cols]
info_df = info_df.sort_values(by = ["experiment", "course", "ID"] )

## Step 5: Fit psychometric functions to the data

This code uses the function
    
    fit_ps
    
so you will be in trouble if you have not run the cell above in which this function is defined. Fitting the psychometric functions takes a few minutes. 

After fitting, the data from all participants are stored in a variable type called a *Pandas dataframe*. This variable type is useful in many ways, including that it makes it easy to save the combined data in a new csv file that can be used statistical analysis or figure making outside of Python. You will not be working with this new csv file in this course, instead you will be grabbing the data directly from the dataframe. 

In [ ]:
fit_params = [ [], [] ]
pse_slope = [ [], [] ]
for d, data_type in enumerate(["same", "small"]):
    print('Fitting psychometric functions for "{}" condition ... '.format(data_type), end='')
    for index, row in info_df.iterrows():
        # same inducers
        fit_data = [ row[x] for x in info_df.columns if x.startswith("testbigger-{0}-".format(data_type)) ]
        n_trials = [ row[x] for x in info_df.columns if x.startswith("ntrials-{0}-".format(data_type)) ]
        temp_params, threshold, slope = fit_ps(test_inner_sizes, fit_data, n_trials)
        fit_params[d].append(temp_params)
        pse_slope[d].append((threshold, slope))
    # run fit on averages for illustration purposes
    fit_data = [ info_df[x].mean() for x in info_df.columns if x.startswith("testbigger-{0}-".format(data_type)) ]
    n_trials = [ int(info_df[x].mean()) for x in info_df.columns if x.startswith("ntrials-{0}-".format(data_type)) ]
    temp_params, threshold, slope = fit_ps(test_inner_sizes, fit_data, n_trials)
    fit_params[d].append(temp_params)
    pse_slope[d].append((threshold, slope))
    # assign pses to info_df
    info_df["pse-{0}".format(data_type)] = [ x[0] for x in pse_slope[d][0:-1] ]
    info_df["pse-{0}-avefit".format(data_type)] = [pse_slope[d][-1][0]] * info_df.shape[0]
    info_df["slope-{0}".format(data_type)] = [ x[1] for x in pse_slope[d][0:-1] ]
    info_df["slope-{0}-avefit".format(data_type)] = [pse_slope[d][-1][1]] * info_df.shape[0]
    print('finished!'.format(data_type))
# save data to a csv
info_df.to_csv("ebbinghaus_combined.csv")

## Assignment 2
### Question 1 (4 pts): 

The code 

    plot_ps(info_df, fit_params, participant_id)
    
creates two plots, one of the reaction time and one of the responses. The input arguments info_df and fit_params should already be on the workspace if you have run steps 1-5 of the code, and the argument participant_id is a string that indicates the participant ID you want to plot. You can also pass "means" to participant_id and the function will plot the average across all participants.

Reaction times and responses are plotted seperately for each of the seven physical sizes used for the inner *test disc* (in pixels): 20, 23, 24, 25, 26, 27, 30. The physical size of the inner *reference disc* was always 25 pixels. The same inducers condition is shown in blue, and the small inducers in orange. 

(a) Please use the function plot_ps to plot the averages across all participants. 

(b) Then use the function plot_ps to plot your own data - input your own participant ID to the function. If your data was excluded you may plot someone else's ID. Participant "999" has very reasonable data. 

(c) Then draw, by hand, S-shaped *Psychometric functions* through the data, and indicate the approximate *Point of Subjective Equality* (PSE). Do this separately for the same and small inducer conditions, in different colors. Do this for both the average data and your own data.  

(d) Based on what you did in 1a-1c, is your effect size, measured using the PSEs, bigger than the average or smaller than the average? Explain why. 

Your answer should include the code used in a and b (2 lines per question, at most) and screenshots of the *Responses* part of the plot with your hand drawn functions.  



In [ ]:
# work on your answer here:

### Question 2 (4 pts): 

The code 

    pse_data = np.array(info_df[["pse-same","pse-small"]])

grabs data from the Pandas dataframe to create a 2 x n numpy array that contains the PSEs for the same and small inducers condition, respectively. 

(a) Please use the *shape* method to get the **number of participants in your dataset** and save that as a integer variable named
    
    num_subs

(b) Please use the methods *mean* and *std* or their corresponding numpy commands to compute the **mean** and **standard deviation**, seperately for the two conditions, and save them as two variables named:

    pse_means
    pse_stdev

Note that both mean and standard deviation can be computed in one line of code.

(c) Watch this [video](https://www.youtube.com/watch?v=AQy11Hfp_dU) (also on eClass) to learn how to compute **standard error** using the sample size, and how to convert standard error to the 95% confidence interval. You can use the command *np.sqrt* to take the square root of a number. Save as two variables named:
    
    pse_stderr # standard error
    pse_ci # interval

Use the following code to plot your data as a bar plot with error bars:
    
    plt.bar((0,1), pse_means, yerr=pse_ci, capsize=5)
    plt.ylim([15,30])
    
Try replacing pse_ci with pse_stderr in the above and observe how the error bars change. 
    
(d) The size of the *Ebbinghaus effect* for each participant can be computed as the difference between pse_same and pse_small. Use pse_data to subtract pse_small from pse_same and assign the output to a new variable called:

    pse_diff

this can be done in one line by selecting the different columns of pse_data. Now use the method *max* or its corresponding numpy command to assign the maximum effect size to a new variable called

    pse_max
    
Bonus: Use the method *argmax* or its corresponding numpy command to get the index of the participant that has the maximum effect size. You can then get the ID of that participant using this command
    
    print(np.array(info_df[["ID"]])[max_id,0])

where max_id is the index. 

Your answer should include the code used in a-d and a screenshot of the bar plot you created in (c). 


In [ ]:
# work on your answer here:
pse_data = np.array(info_df[["pse-same","pse-small"]])

## Question 3 (2 pts):

According to the article “The surface area of human V1 predicts the subjective experience of object size”, linked on eClass, what would you expect to be true about primary visual cortex (area V1) of those participants who have the largest Ebbinghaus effects in your experiment?